# Draw with IPyCanvas

[![Open in Colab](https://lab.aef.me/files/assets/colab-badge.svg)](https://colab.research.google.com/github/adamelliotfields/lab/blob/main/files/draw.ipynb)
[![Open in Kaggle](https://lab.aef.me/files/assets/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/adamelliotfields/lab/blob/main/files/draw.ipynb)
[![Render nbviewer](https://lab.aef.me/files/assets/nbviewer_badge.svg)](https://nbviewer.org/github/adamelliotfields/lab/blob/main/files/draw.ipynb)
[![Try lite now](https://lab.aef.me/files/assets/badge.svg)](https://lab.aef.me/lab/?path=draw.ipynb)

Drawing app with `ipywidgets` and `ipycanvas`. Originally based on [examples/hand_drawing.ipynb](https://github.com/martinRenou/ipycanvas/blob/master/examples/hand_drawing.ipynb).

In [ ]:
import threading

from ipycanvas import RoughCanvas, hold_canvas
from ipywidgets import Button, Checkbox, ColorPicker, HBox, IntSlider, VBox, link

In [ ]:
def debounce(wait):
    def decorator(fn):
        def debounced(*args, **kwargs):
            def call_fn():
                debounced._timer = None
                return fn(*args, **kwargs)

            if debounced._timer is not None:
                debounced._timer.cancel()
            debounced._timer = threading.Timer(wait, call_fn)
            debounced._timer.start()

        debounced._timer = None
        return debounced

    return decorator


def on_mouse_down(x, y):
    global drawing, position, shape
    drawing = True
    position = (x, y)
    shape = [position]


def on_mouse_move(x, y):
    global position
    if not drawing or position is None:
        return

    with hold_canvas():
        canvas.stroke_line(position[0], position[1], x, y)
        position = (x, y)

    shape.append(position)


def on_mouse_up(x, y):
    global drawing, fill_checkbox, position, shape
    if position is None:
        return

    with hold_canvas():
        canvas.stroke_line(position[0], position[1], x, y)
        if fill_checkbox.value:
            canvas.fill_polygon(shape)

    drawing = False
    shape = []


@debounce(0.15)
def update_canvas_size(_):
    # note that updating the canvas size will clear the canvas
    global canvas, height_slider, width_slider
    canvas.width = width_slider.value
    canvas.height = height_slider.value

In [ ]:
# state
drawing = False
position = None
shape = []

# initial canvas size
default_width = 640
default_height = 480

# initial colors
default_stroke_color = "#749cb8"
default_fill_color = "#c4d8e2"

canvas = RoughCanvas(width=default_width, height=default_height, sync_image_data=True)
canvas.layout.border = "1px solid black"  # border isn't included in image data

canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

stroke_picker = ColorPicker(description="Stroke color:", value=default_stroke_color)
fill_picker = ColorPicker(description="Fill color:", value=default_fill_color)
fill_checkbox = Checkbox(description="Fill?", value=False)

link((stroke_picker, "value"), (canvas, "stroke_style"))
link((fill_picker, "value"), (canvas, "fill_style"))

width_slider = IntSlider(description="Width:", value=default_width, min=256, max=1024)
height_slider = IntSlider(description="Height:", value=default_height, min=256, max=1024)

width_slider.observe(update_canvas_size, names="value")
height_slider.observe(update_canvas_size, names="value")

clear_button = Button(description="Clear", button_style="danger")
clear_button.layout.width = "fit-content"
clear_button.on_click(lambda _: canvas.clear())

# TODO: use an observer to store in a numpy array which can be used to save to disk or input to a model
# TODO: canvas background color picker above stroke color picker (default is transparent)
# TODO: toggle rough style (Canvas vs RoughCanvas constructors)
left_box = VBox([stroke_picker, fill_picker, fill_checkbox, clear_button])
right_box = VBox([width_slider, height_slider])
VBox([HBox([canvas]), HBox([left_box, right_box])])